In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize 
from gensim.models import Word2Vec
import warnings 
warnings.filterwarnings(action = 'ignore') 

In [2]:
pageview = pd.read_csv('pageview.csv', error_bad_lines = False)
pageview.shape

b'Skipping line 1322258: expected 4 fields, saw 5\n'


(2749999, 4)

In [3]:
pageview.head()

,ROW_NUM,USER_ID,URL_PATH,CENSUS_KEY
0,2750000,5150228a-ad62-4a14-9169-79e7979e1a5e_46ac5dea-...,/en/friends-feelings/common-challenges/self-co...,58a9cf6a144233ad9c286a2533872b2c3a9c1998
1,2749999,5150228a-ad62-4a14-9169-79e7979e1a5e_f13c5ba7-...,/en/school-learning/partnering-with-childs-sch...,79cfc7a3c90fa4fc002ec3978716f823078afc23
2,2749998,5150228a-ad62-4a14-9169-79e7979e1a5e_22e91c69-...,/en/learning-attention-issues/child-learning-d...,bc2e78278714e173c09535d7d0260e6299b9bd09
3,2749997,5150228a-ad62-4a14-9169-79e7979e1a5e_5fe37e08-...,/en/learning-attention-issues/child-learning-d...,07e692d47d057cb1d0409cf6e701f9be5cda3e36
4,2749996,5150228a-ad62-4a14-9169-79e7979e1a5e_f106f359-...,/en/learning-attention-issues/child-learning-d...,85e32c288526fc2b24b0db665254c7f2b433a14e


In [4]:
census = pd.read_csv('census.csv', error_bad_lines = False)
census.head()

,NORTH_SOUTH,CENSUS_KEY,TIMEZONE,DIFF_FROM_US_MEAN_INCOME,PCT_TOTAL_URBAN_HH,PCT_TOTAL_RURAL_HH,PCT_BELOW_POVERTY_LEVEL,PCT_UNDER_5_YRS_AGE,PCT_SPEAKS_LANGUAGE_OTHER_THAN_ENGLISH,AVG_HH_SIZE,...,PCT_TOTAL_35_TO_44_BACHELORS,PCT_TOTAL_45_TO_64_BACHELORS,PCT_TOTAL_65_AND_ABOVE_BACHELORS,PCT_TOTAL_HISP_OR_LATINX,PCT_TOTAL_WHITE_NOT_HISP_OR_LATINX,PCT_TOTAL_WHITE_ALONE,PCT_TOTAL_BLACK_OR_AA_ALONE,PCT_TOTAL_AMERICAN_INDIAN_AND_ALASKA_NATIVE_ALONE,PCT_TOTAL_ASIAN_ALONE,PCT_TOTAL_NATIVE_HI_AND_PACIFIC_ISLANDER_ALONE
0,NORTH,026fadfabedc8fb74b6541384357664505c7ce45,America/New York,-7187,0.205,-0.205,-5.3,-1.4,-4.5,-0.35,...,0.011,0.004,0.004,-0.110,0.264,0.197,-0.114,-0.008,-0.023,0.000
1,NORTH,bb0fce0a981352d43b40f5b844859ebdc246c705,America/New York,-979,0.069,-0.069,16.7,-3.2,-3.4,-0.13,...,-0.003,0.044,0.04,-0.113,0.119,0.045,-0.071,-0.002,0.063,-0.001
2,NORTH,423954bfe8bae9f4faa66eda54301e9aebff4c9a,America/New York,-28,-0.79,0.79,-7.7,1,-16.4,0,...,-0.008,-0.013,-0.019,-0.150,0.345,0.252,-0.121,-0.008,-0.052,-0.002
3,NORTH,c6a78684b871baa185bd835e6a5ee14790d04ad5,America/New York,-4524,0.092,-0.092,-4.6,-2.7,-12.6,-0.37,...,0.004,0.04,0.008,-0.123,0.275,0.196,-0.104,-0.008,-0.036,-0.002
4,NORTH,bce53624912fa698b3df3b3a7a11bf9ba6a0eeb3,America/New York,19681,0.185,-0.185,-10.1,-1.5,-12.9,0,...,0.01,0.058,0.027,-0.153,0.254,0.159,-0.077,-0.007,-0.008,-0.002


In [5]:
content = pd.read_csv('content.csv', error_bad_lines = False)
content.head()

,url,entry_type,title,article_content,meta_title,meta_description,meta_keywords
0,/g26ftzdilh/bfunderstanding-dyslexia,Deep Dive Article,Understanding Dyslexia,Raising a child with dyslexia can stir up a lo...,Understanding Dyslexia in Children | Dyslexia ...,Have you asked yourself what is dyslexia? Lear...,"dyslexia, what is dyslexia, signs of dyslexia,..."
1,/g26ftzdilh/blazingly-fast-understanding-dyslexia,Deep Dive Article,Blazingly Fast Understanding Dyslexia,Raising a child with dyslexia can stir up a lo...,Entender la dislexia | ¿Qué es la discapacidad...,La dislexia es una condición real del cerebro ...,"Definición de la dislexia, qué es la dislexia,..."
2,/learning-attention-issues/getting-started/wha...,Deep Dive Article,Awareness Month 2018: Supporting the 1 in 5,When it comes to learning and attention issues...,Awareness Month 2018: Supporting the 1 in 5,Understood is celebrating ADHD and learning di...,"learning disability awareness, adhd awareness,..."
3,/learning-attention-issues/getting-started/wha...,Deep Dive Article,"Helping Your Child, and You, #BeUnderstood","October is Learning Disabilities, ADHD, and Dy...","Helping Your Child, and You, #BeUnderstood",When kids with learning and attention issues a...,"BeUnderstood, #BeUnderstood, Be Understood, Un..."
4,/learning-attention-issues/child-learning-disa...,Deep Dive Article,Understanding ADHD,What is ADHD? ADHD is a common condition that ...,What Is ADHD,"What is ADHD, and how does it affect kids? Use...","adhd, add, what is adhd, whats adhd, define ad..."


In [15]:
import string
from nltk.corpus import stopwords 
nltk.download('stopwords')

# select features from original dataset
cotent_texts = content[['title', 'article_content', 'meta_title', 'meta_description', 'meta_keywords']]
content['merged'] = cotent_texts.astype(str).apply(' '.join, 1).str.lower()

# remove punctuations
content['merged'] = content['merged'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# remove stopwords
stop_words = stopwords.words('english')
content['merged_words'] = content['merged'].apply(lambda x: [word for word in x.split(' ') if not word in stop_words])

# content['merged_words'] = content['merged'].apply(lambda x: x.split(' '))
content.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/soojinkim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,url,entry_type,title,article_content,meta_title,meta_description,meta_keywords,merged,merged_words
0,/g26ftzdilh/bfunderstanding-dyslexia,Deep Dive Article,Understanding Dyslexia,Raising a child with dyslexia can stir up a lo...,Understanding Dyslexia in Children | Dyslexia ...,Have you asked yourself what is dyslexia? Lear...,"dyslexia, what is dyslexia, signs of dyslexia,...",understanding dyslexia raising a child with dy...,"[understanding, dyslexia, raising, child, dysl..."
1,/g26ftzdilh/blazingly-fast-understanding-dyslexia,Deep Dive Article,Blazingly Fast Understanding Dyslexia,Raising a child with dyslexia can stir up a lo...,Entender la dislexia | ¿Qué es la discapacidad...,La dislexia es una condición real del cerebro ...,"Definición de la dislexia, qué es la dislexia,...",blazingly fast understanding dyslexia raising ...,"[blazingly, fast, understanding, dyslexia, rai..."
2,/learning-attention-issues/getting-started/wha...,Deep Dive Article,Awareness Month 2018: Supporting the 1 in 5,When it comes to learning and attention issues...,Awareness Month 2018: Supporting the 1 in 5,Understood is celebrating ADHD and learning di...,"learning disability awareness, adhd awareness,...",awareness month 2018 supporting the 1 in 5 whe...,"[awareness, month, 2018, supporting, 1, 5, com..."
3,/learning-attention-issues/getting-started/wha...,Deep Dive Article,"Helping Your Child, and You, #BeUnderstood","October is Learning Disabilities, ADHD, and Dy...","Helping Your Child, and You, #BeUnderstood",When kids with learning and attention issues a...,"BeUnderstood, #BeUnderstood, Be Understood, Un...",helping your child and you beunderstood octobe...,"[helping, child, beunderstood, october, learni..."
4,/learning-attention-issues/child-learning-disa...,Deep Dive Article,Understanding ADHD,What is ADHD? ADHD is a common condition that ...,What Is ADHD,"What is ADHD, and how does it affect kids? Use...","adhd, add, what is adhd, whats adhd, define ad...",understanding adhd what is adhd adhd is a comm...,"[understanding, adhd, adhd, adhd, common, cond..."


In [23]:
"""
size: The number of dimensions of the embeddings and the default is 100.
window: The maximum distance between a target word and words around the target word. The default window is 5.
min_count: The minimum count of words to consider when training the model; 
           words with occurrence less than this count will be ignored. 
           The default for min_count is 5.
workers: The number of partitions during training and the default workers is 3.
sg: The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW.
"""

word2vec = Word2Vec(content['merged_words'], min_count=3, size=32, workers=3, window=3, sg=0) # CBOW
print(word2vec.similarity('dyslexia', 'adhd'))
print(word2vec.similarity('dyslexia', 'understanding'))

0.45340493
0.33444974


In [24]:
word_list = list(word2vec.wv.vocab.keys())
word_embeddings = pd.Series(word_list, index=word_list).apply(lambda x: word2vec[x])
df = pd.DataFrame.from_dict(word_embeddings.to_dict()).T
colnames = ['embed_' + str(i) for i in range(32)]
df.columns = colnames
df.to_csv('word2vec_stopwords_removed')